In [ ]:
!pip install ultralytics lapx shapely -q

In [ ]:
import cv2
from ultralytics import YOLO
from collections import defaultdict
from ultralytics.solutions import object_counter
from shapely.geometry import Point, Polygon
from ultralytics.utils.checks import check_imshow, check_requirements
from ultralytics.utils.plotting import Annotator, colors
check_requirements("shapely>=2.0.0")

In [ ]:
class QueueManager:
    """A class to manage the queue management in real-time video stream based on their tracks."""

    def __init__(self):
        """Initializes the queue manager with default values for various tracking and counting parameters."""

        # Mouse events
        self.is_drawing = False
        self.selected_point = None

        # Region & Line Information
#         self.reg_pts = [(20, 60), (20, 680), (1120, 680), (1120, 60)]
        self.counting_region = None
        self.region_color = (255, 0, 255)
        self.region_thickness = 5

        # Image and annotation Information
        self.im0 = None
        self.tf = None
        self.view_img = False
        self.view_queue_counts = True
        self.fontsize = 0.6

        self.names = None  # Classes names
        self.annotator = None  # Annotator
        self.window_name = "Ultralytics YOLOv8 Queue Manager"

        # Object counting Information
        self.counts = 0
        self.count_txt_color = (255, 255, 255)

        # Tracks info
        self.track_history = defaultdict(list)
        self.track_thickness = 2
        self.draw_tracks = False
        self.track_color = None

        # Check if environment support imshow
        self.env_check = check_imshow(warn=True)

    def set_args(
        self,
        classes_names,
        reg_pts,
        line_thickness=2,
        track_thickness=2,
        view_img=True,
        region_color=(255, 0, 255),
        view_queue_counts=True,
        draw_tracks=True,
        count_txt_color=(255, 255, 255),
        track_color=(0,128,0),
        region_thickness=5,
        fontsize=0.7,
    ):
        """
        Configures the Counter's image, bounding box line thickness, and counting region points.

        Args:
            line_thickness (int): Line thickness for bounding boxes.
            view_img (bool): Flag to control whether to display the video stream.
            view_queue_counts (bool): Flag to control whether to display the counts on video stream.
            reg_pts (list): Initial list of points defining the counting region.
            classes_names (dict): Classes names
            region_color (RGB color): Color of queue region
            track_thickness (int): Track thickness
            draw_tracks (Bool): draw tracks
            count_txt_color (RGB color): count text color value
            track_color (RGB color): color for tracks
            region_thickness (int): Object counting Region thickness
            fontsize (float): Text display font size
        """
        self.tf = line_thickness
        self.view_img = view_img
        self.view_queue_counts = view_queue_counts
        self.track_thickness = track_thickness
        self.draw_tracks = draw_tracks
        self.region_color = region_color

        if len(reg_pts) >= 3:
            print("Queue region initiated...")
            self.reg_pts = reg_pts
            self.counting_region = Polygon(self.reg_pts)
        else:
            print("Invalid region points provided...")
            print("Using default region now....")
            self.counting_region = Polygon(self.reg_pts)

        self.names = classes_names
        self.track_color = track_color
        self.count_txt_color = count_txt_color
        self.region_thickness = region_thickness
        self.fontsize = fontsize

    def extract_and_process_tracks(self, tracks):
        """Extracts and processes tracks for queue management in a video stream."""

        # Annotator Init and queue region drawing
        self.annotator = Annotator(self.im0, self.tf, self.names)

        if tracks[0].boxes.id is not None:
            boxes = tracks[0].boxes.xyxy.cpu()
            clss = tracks[0].boxes.cls.cpu().tolist()
            track_ids = tracks[0].boxes.id.int().cpu().tolist()

            # Extract tracks
            for box, track_id, cls in zip(boxes, track_ids, clss):
                # Draw bounding box
                self.annotator.box_label(box, label=f"{self.names[cls]}#{track_id}", color=colors(int(track_id), True))
                
                # Use top point instead of centroid
                top_point = (int((box[0] + box[2]) / 2), int(box[1]))
                # bottom_point = (int((box[0] + box[2]) / 2), int(box[3]))
                
                # Draw Tracks
                track_line = self.track_history[track_id]
#                 track_line.append((float((box[0] + box[2]) / 2), float((box[1] + box[3]) / 2)))
                track_line.append(top_point)
                if len(track_line) > 30:
                    track_line.pop(0)

                # Draw track trails
                if self.draw_tracks:
                    self.annotator.draw_centroid_and_tracks(
                        track_line,
                        color=self.track_color if self.track_color else colors(int(track_id), True),
                        track_thickness=self.track_thickness,
                    )

                prev_position = self.track_history[track_id][-2] if len(self.track_history[track_id]) > 1 else None

                if len(self.reg_pts) >= 3:
                    is_inside = self.counting_region.contains(Point(track_line[-1]))
                    if prev_position is not None and is_inside:
                        self.counts += 1
            if self.counts > 7:
                # print("Warning: Queue count exceeded.")
                cv2.putText(self.im0, "Warning: Queue count exceeded!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA) 

        label = "Queue Counts : " + str(self.counts)

        if label is not None:
            self.annotator.queue_counts_display(
                label,
                points=self.reg_pts,
                region_color=self.region_color,
                txt_color=self.count_txt_color,
                fontsize=self.fontsize,
            )

        self.counts = 0
        self.display_frames()

    def display_frames(self):
        """Display frame."""
        if self.env_check:
            self.annotator.draw_region(reg_pts=self.reg_pts, thickness=self.region_thickness, color=self.region_color)
            cv2.namedWindow(self.window_name)
            cv2.imshow(self.window_name, self.im0)
#             Break Window
            if cv2.waitKey(1) & 0xFF == ord("q"):
                return

    def process_queue(self, im0, tracks):
        """
        Main function to start the queue management process.

        Args:
            im0 (ndarray): Current frame from the video stream.
            tracks (list): List of tracks obtained from the object tracking process.
        """
        self.im0 = im0  # store image
        self.extract_and_process_tracks(tracks)  # draw region even if no objects

        if self.view_img:
            self.display_frames()
        return self.im0
    
if __name__ == "__main__":
    QueueManager()

In [ ]:
from tqdm import tqdm
# from ultralytics.solutions import queue_management
model = YOLO("yolov8x.pt")
cap = cv2.VideoCapture("/kaggle/input/final-ticketing/Ticketing_Line_2_Cam3_1.avi")

assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH,
                                       cv2.CAP_PROP_FRAME_HEIGHT,
                                       cv2.CAP_PROP_FPS))

video_writer = cv2.VideoWriter("queue_management.avi",
                               cv2.VideoWriter_fourcc(*'mp4v'),
                               fps,
                               (w, h))

#queue_region = [[235,911],[179,381],[681,249],[1038,213],[1038,213],[1233,179],[1233,316],[930,554],[674,849]]
#235,911,179,381,681,249,1038,213,1233,179,1233,316,930,554,674,849" shape="poly">
classes_to_count = [0,1]

#queue = QueueManager()
# #queue.set_args(classes_names=model.names,
#                reg_pts=queue_region,
#                line_thickness=3,
#                fontsize=1.0,
#                region_color=(255, 144, 31))


# # Main loop for processing video frames
# while cap.isOpened():
#     success, im0 = cap.read()

#     if success:
#         # Object detection and tracking
#         tracks = model.track(im0, show=False, persist=True,
#                              verbose=False, classes=0)  # Only person class
#         out = queue.process_queue(im0, tracks)

#         # Write frame to video
#         video_writer.write(im0)

#     else:
#         print("Video processing has been successfully completed.")
#         break

# # Release resources
# cap.release()
# video_writer.release()
# # cv2.destroyAllWindows()
# Init Object Counter

counter = object_counter.ObjectCounter()
counter.set_args(view_img=True,
                 reg_pts=queue_region,
                 classes_names=model.names,
                 draw_tracks=True,
                 line_thickness=2)

# Get total number of frames
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Main loop for processing video frames with tqdm
for _ in tqdm(range(total_frames)):
    success, im0 = cap.read()

    if success:
        # Object detection and tracking
        tracks = model.track(im0, show=False, persist=True,
                             verbose=False, classes=0, imgsz=4096)  # Only person class
        out = queue.process_queue(im0, tracks)

        # Write frame to video
        video_writer.write(im0)

    else:
        print("Video processing has been successfully completed.")
        break

# Release resources
cap.release()
video_writer.release()
# cv2.destroyAllWindows()

In [ ]:
import cv2

def trim_video(input_path, output_path, start_time, end_time):
    # Open input video file
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print("Error: Unable to open input video file")
        return

    # Get frame rate and total number of frames
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate start and end frame numbers
    start_frame = int(start_time * fps)
    end_frame = int(end_time * fps)
    
    # Validate start and end frame numbers
    if start_frame >= total_frames:
        print("Error: Start time exceeds video duration")
        return
    if end_frame >= total_frames:
        end_frame = total_frames - 1
    
    # Set start frame
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    # Create video writer
    fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Change codec to XVID for .avi files
    out = cv2.VideoWriter(output_path, fourcc, fps, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

    # Read and write frames within the specified time range
    for frame_num in range(start_frame, end_frame + 1):
        success, frame = cap.read()
        if success:
            out.write(frame)
        else:
            break

    # Release resources
    cap.release()
    out.release()

    print("Video trimmed successfully")


# Input video file path
input_path = "/kaggle/input/pune-metro-hackathon/dataset/ticketing-crowd/1 (6).avi "

# Output video file path
output_path = "/kaggle/working/"

# Input start and end times for trimming (in seconds)
start_time = float(input("Enter the start time (in seconds):"))
end_time = float(input("Enter the end time (in seconds): "))

# Call function to trim the video
trim_video(input_path, output_path, start_time, end_time)


In [ ]:
import cv2
from ultralytics import YOLO, solutions

model = YOLO("yolov8x.pt")
cap = cv2.VideoCapture("/kaggle/input/final-ticketing/Ticketing_Line_2_Cam3_1.avi")
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

#line_points = [(20, 400), (1080, 400)]  # line or region points
classes_to_count = [0, 2]  # person and car classes for count

# Video writer
video_writer = cv2.VideoWriter("object_counting_output.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# Init Object Counter
counter = solutions.ObjectCounter(
    view_img=True,
    classes_names=model.names,
    draw_tracks=True,
    line_thickness=2,
)

while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break
    tracks = model.track(im0, persist=True, show=False, classes=classes_to_count, imgsz=7680)

    im0 = counter.start_counting(im0, tracks)
    video_writer.write(im0)

cap.release()
video_writer.release()
cv2.destroyAllWindows()

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="GZfR2Cnq6cLeRcwf1pxa")
project = rf.workspace("things-xuzgk").project("1-bfedx")
version = project.version(6)
dataset = version.download("yolov8-obb")

In [6]:
!pip install -U ipywidgets -q

In [1]:
!pip install --upgrade wandb -q

In [ ]:
import torch
torch.cuda.empty_cache()

In [2]:
import wandb

In [4]:
wandb.login(key="cf4b308c1d949dd100d0264e19f74f57d611a6ee")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
from ultralytics import YOLO


# Load a model
model = YOLO("yolov8x.pt")  # load a pretrained model (recommended for training)

# Train the model with 2 GPUs
results = model.train(data="/kaggle/working/1-6/data.yaml", epochs=100, imgsz=960, device=[0, 1])

In [3]:
!pip install ultralytics==8.2.18

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 756.7 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.2/757.2 kB 6.2 MB/s eta 0:00:00a 0:00:01


In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("/kaggle/working/runs/detect/train18/weights/last.pt")  # load a partially trained model

# Resume training
results = model.train(resume=True)

Ultralytics YOLOv8.2.18 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
                                                      CUDA:1 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/working/runs/detect/train18/weights/last.pt, data=/kaggle/working/1-6/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=960, save=True, save_period=-1, cache=False, device=[0, 1], workers=8, project=None, name=train18, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=/kaggle/working/runs/detect/train18/weights/last.pt, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=Fa

wandb: Currently logged in as: ankit-das16381 (sakec). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.17.0
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240520_074113-2fo4ykls
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run train18
wandb: ⭐️ View project at https://wandb.ai/sakec/YOLOv8
wandb: 🚀 View run at https://wandb.ai/sakec/YOLOv8/runs/2fo4ykls


Transferred 595/595 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /kaggle/working/1-6/train/labels.cache... 4012 images, 32 backgrounds, 0 corrupt: 100%|██████████| 4012/4012 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...

     87/100      15.4G     0.6944     0.4844      1.053         34        960: 100%|██████████| 251/251 [06:03<00:00,  1.45s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 74/74 [00:54<00:00,  1.37it/s]


                   all       1176       4726      0.795       0.72      0.775      0.532

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     88/100      15.3G     0.6928      0.485       1.06         29        960: 100%|██████████| 251/251 [06:03<00:00,  1.45s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 74/74 [00:54<00:00,  1.37it/s]


                   all       1176       4726      0.798      0.707      0.775      0.533

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     89/100      15.5G     0.6942     0.4866       1.06         45        960: 100%|██████████| 251/251 [06:03<00:00,  1.45s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  96%|█████████▌| 71/74 [00:52<00:02,  1.41it/s]

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("/kaggle/working/runs/detect/train18/weights/last.pt")  # load a partially trained model

# Resume training
results = model.train(resume=True, data="/kaggle/working/1-6/data.yaml", epochs=100, imgsz=960, device=[0, 1])

In [ ]:
import torch
torch.cuda.device_count()

In [ ]:
# Import libraries (replace 'yaml' with 'pyyaml' if needed)
import yaml

# Define path to the YAML file (replace with your actual path)
yaml_file_path = "/kaggle/working/1-6/data.yaml"

In [ ]:
# Read the YAML content
with open(yaml_file_path, 'r') as stream:
    config = yaml.safe_load(stream)
print(config)

In [ ]:




# Modify desired configurations (replace 'key' and 'value' with your changes)
config['valid'] = 'valid/images'

# Write the modified configuration back to the YAML file
with open(yaml_file_path, 'w') as stream:
    yaml.dump(config, stream)

print(f"YAML file '{yaml_file_path}' successfully modified!")
